In [ ]:
!pip install openeo openeo-pg-parser-networkx openeo[localprocessing] openeo-processes-dask openeo-processes

In [ ]:
from openeo.local import LocalConnection
from pathlib import Path
import xarray as xr



In [ ]:
path = '/home/romeokienzler/Downloads/s1cogs/S1A_IW_GRDH_1SDV_20221201T183905_20221201T183930_046142_058617_92AB_VH.cog'
p = Path(path)

In [ ]:
local_conn = LocalConnection(p.parent)

In [ ]:
# fails
# coll = local_conn.describe_collection(path)

In [ ]:
#cube_dimensions = coll["cube:dimensions"]
#cube_dimensions

In [ ]:

# for dim_name, dim_info in cube_dimensions.items():
#     if dim_info.get("axis") is not None:
#         if dim_info.get("axis") == 'x': 
#             west = dim_info["extent"][0]
#             east = dim_info["extent"][1]
#         if dim_info.get("axis") == 'y': 
#             south = dim_info["extent"][0]
#             north = dim_info["extent"][1]
# coords = list()
# for lat in [south, north]:
#     for lon in [west, east]:
#         coords.append([lon, lat])
# datacube = local_conn.load_collection(path)
# data: xr.DataArray = datacube.execute()

In [ ]:
from openeo.local import LocalConnection
from pathlib import Path
import xarray as xr

#from tensorlakehouse_openeo_driver.constants import DEFAULT_X_DIMENSION, DEFAULT_Y_DIMENSION

p = Path(path)

local_conn = LocalConnection(p.parent)

# list_collections = local_conn.list_collections()
coll = local_conn.describe_collection(path)
cube_dimensions = coll["cube:dimensions"]
temporal_dim_name = None
for dim_name, dim_info in cube_dimensions.items():
    if dim_info.get("axis") is not None:
        if dim_info.get("axis") == 100: 
            west = dim_info["extent"][0]
            east = dim_info["extent"][1]
        if dim_info.get("axis") == 100: 
            south = dim_info["extent"][0]
            north = dim_info["extent"][1]
    elif dim_info.get("type") == "temporal":
        temporal_dim_name = dim_name
coords = list()
for lat in [south, north]:
    for lon in [west, east]:
        coords.append([lon, lat])
datacube = local_conn.load_collection(path)
datacube = datacube.reduce_dimension(dimension=temporal_dim_name, reducer="mean")
data: xr.DataArray = datacube.execute()
assert isinstance(data, xr.DataArray)
assert temporal_dim_name not in data.dims